In [1]:
import tensorflow as tf
from keras import backend as K
import numpy as np
from keras.layers import Embedding
# print(tf.__version__)

Using TensorFlow backend.


In [2]:
def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [3]:
hidden_size=256
vb_size=18
batchsize=2
weights = {
    'W_conv1': tf.get_variable('W0', shape=(3,3,3,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'W_conv2': tf.get_variable('W1', shape=(3,3,32,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'W_conv3': tf.get_variable('W2', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'W_conv4': tf.get_variable('W3', shape=(3,3,64,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'W_conv5': tf.get_variable('W4', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'W_conv6': tf.get_variable('W5', shape=(3,3,128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'W_fc1': tf.get_variable('W6', shape=(28*28*128,1024), initializer=tf.contrib.layers.xavier_initializer()), 
    'W_fc2': tf.get_variable('W7', shape=(1024,50), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W8', dtype = tf.float64,shape=(hidden_size,vb_size), initializer=tf.contrib.layers.xavier_initializer()), 
    }
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc4': tf.get_variable('B3', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc5': tf.get_variable('B4', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bc6': tf.get_variable('B5', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'b_fc1': tf.get_variable('B6', shape=(1024), initializer=tf.contrib.layers.xavier_initializer()),
    'b_fc2': tf.get_variable('B7', shape=(50), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B8', dtype = tf.float64,shape=(vb_size), initializer=tf.contrib.layers.xavier_initializer()),
    }

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [107]:
class GRU_cell(object):

    def __init__(self, input_nodes, hidden_unit, output_nodes):

        self.input_nodes = input_nodes
        self.hidden_unit = hidden_unit
        self.output_nodes = output_nodes

        self.Wx = tf.Variable(tf.zeros([self.input_nodes, self.hidden_unit]))

        self.Wr = tf.Variable(tf.zeros([self.input_nodes, self.hidden_unit]))
        self.br = tf.Variable(tf.truncated_normal([self.hidden_unit], mean=1))
        
        self.Wz = tf.Variable(tf.zeros([self.input_nodes, self.hidden_unit]))
        self.bz = tf.Variable(tf.truncated_normal([self.hidden_unit], mean=1))

        self.Wh = tf.Variable(tf.zeros([self.hidden_unit, self.hidden_unit]))

        self.Wo = tf.Variable(tf.truncated_normal([self.hidden_unit, self.output_nodes], mean=1, stddev=.01))
        self.bo = tf.Variable(tf.truncated_normal([self.output_nodes], mean=1, stddev=.01))

        self.images = tf.placeholder(tf.float32,shape=[None, self.input_nodes/2], name='images')
        self.words = tf.placeholder(tf.int32,shape=[None, None], name='words')
#         batch_input_ = tf.transpose(self._inputs, perm=[2, 0, 1])
#         self.processed_input = tf.transpose(batch_input_)

#         batch_input_ = tf.transpose(self._inputs, perm=[2, 0, 1])
#         self.processed_input = tf.transpose(batch_input_)

        VOCAB_LEN=19
        EMBED_SIZE=50
        embeddings =tf.Variable(tf.random_uniform([VOCAB_LEN, EMBED_SIZE]))
#         word_embed = tf.placeholder(dtype=tf.int32, shape=(None,None), name='caption_p')
        embed = tf.nn.embedding_lookup(embeddings, self.words)
        self.processed_input = embed

        self.initial_hidden = self.processed_input[:, 0, :]
        self.initial_hidden = tf.matmul(self.initial_hidden, tf.zeros([input_nodes/2, hidden_unit]))
#         self.initial_hidden = tf.zeros([None,hidden_unit])
# 
    def Gru(self, previous_hidden_state, x):
        x = K.expand_dims(x,0)
        x = K.expand_dims(x,0)
        features_try = K.tile(K.expand_dims(self.images, 1), [1, K.shape(x)[1], 1])
        print(features_try.shape,x.shape)
        embeddings = tf.concat([features_try,x],2)
        print("embed ip:",embeddings.shape)
        
        z = tf.sigmoid(tf.matmul(embeddings[0], self.Wz) + self.bz)
        r = tf.sigmoid(tf.matmul(embeddings[0], self.Wr) + self.br)

        h_ = tf.tanh(tf.matmul(embeddings[0], self.Wx) +
                     tf.matmul(previous_hidden_state, self.Wh) * r)

        current_hidden_state = tf.multiply( (1 - z), h_) + tf.multiply(previous_hidden_state, z)

        return current_hidden_state

    def get_states(self):
#         all_hidden_states=[]
#         prev_hidden_state = self.initial_hidden
#         for inp in self.processed_input:
#             features_try = K.tile(K.expand_dims(self.inputs, 1), [1, K.shape(embed)[1], 1])
#             embeddings = tf.concat([features_try,embed],2)
#             prev_hidden_state = self.Gru(prev_hidden_state,embeddings)
#             all_hidden_states.append(prev_hidden_state)
#         for i in range(len(self.processed_input)):
        print("%%",self.processed_input[0].shape,self.initial_hidden.shape)
        x=self.processed_input[0]
        all_hidden_states = tf.scan(self.Gru, x, initializer=self.initial_hidden, name='states')
        print("all_ht:",all_hidden_states.shape)
        return all_hidden_states

    def get_output(self, hidden_state):
        output = tf.nn.relu(tf.matmul(hidden_state, self.Wo) + self.bo)
        print("op:",output.shape)
        return output

    def get_outputs(self):
        all_hidden_states = self.get_states()
        all_outputs = tf.map_fn(self.get_output, all_hidden_states)
        print("all_op:",all_outputs.shape)
        return all_outputs
    
    def get_last_output(self):
        all_ht = self.get_states()
        last_ht = all_ht[-1]
        return(self.get_output(last_ht))

In [5]:
epsilon = 1e-3

In [6]:
def batch_norm_wrapper(inputs, is_training, decay = 0.999):

    scale = tf.Variable(tf.ones([inputs.get_shape()[-1]]))
    beta = tf.Variable(tf.zeros([inputs.get_shape()[-1]]))
    pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]), trainable=False)
    pop_var = tf.Variable(tf.ones([inputs.get_shape()[-1]]), trainable=False)

    if is_training:
        batch_mean, batch_var = tf.nn.moments(inputs,[0])
        train_mean = tf.assign(pop_mean,
                               pop_mean * decay + batch_mean * (1 - decay))
        train_var = tf.assign(pop_var,
                              pop_var * decay + batch_var * (1 - decay))
        with tf.control_dependencies([train_mean, train_var]):
            return tf.nn.batch_normalization(inputs,batch_mean, batch_var, beta, scale, epsilon)
    else:
        return tf.nn.batch_normalization(inputs,pop_mean, pop_var, beta, scale, epsilon)

In [7]:
# x = tf.placeholder("float", [None,28,28,1])
# y = tf.placeholder("float", [None, n_classes])
def cnn(x,weights,biases):
    print("in cnn")
    
    '''
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,1,32])),#56
               'W_conv2':tf.Variable(tf.random_normal([3,3,32,32])),#56
               'W_conv3':tf.Variable(tf.random_normal([3,3,32,64])),#28
               'W_conv4':tf.Variable(tf.random_normal([3,3,64,64])),#28
               'W_conv5':tf.Variable(tf.random_normal([3,3,64,128])),#14
               'W_conv6':tf.Variable(tf.random_normal([3,3,128,128])),#14
               'W_fc1':tf.Variable(tf.random_normal([7*7*128,1024])),  # since 3 times maxpooling.. inputsize/2^3
               'W_fc2':tf.Variable(tf.random_normal([1024,1024]))
              }
                  # depending on what that repeat vector does

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([32])),
               'b_conv3':tf.Variable(tf.random_normal([64])),
               'b_conv4':tf.Variable(tf.random_normal([64])),
               'b_conv5':tf.Variable(tf.random_normal([128])),
               'b_conv6':tf.Variable(tf.random_normal([128])),
               'b_fc1':tf.Variable(tf.random_normal([1024])),
               'b_fc2':tf.Variable(tf.random_normal([1024]))
             }
    '''
    
    print("-1")
#     x = tf.convert_to_tensor(x)
    print("00")
    print("bef",x.shape)
    x = tf.reshape(x, shape=[-1, 224, 224, 3])
    print("aft",x.shape)
    print("0")
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1'])+  biases['bc1'])
    print("********",weights['W_conv1'])
    print("1")
    print("conv1:",conv1.shape)
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['bc2'])
    print("2")
    print("conv2:",conv2.shape)
    conv2 = maxpool2d(conv2)
    print("3")
    print("maxpool:",conv2.shape)
#     conv2 = tf.nn.dropout(conv2, 0.25)
#     print("dropout:",conv2.shape)
    print("okay")
    
    conv3 = tf.nn.relu(conv2d(conv2, weights['W_conv3']) + biases['bc3'])
    print("conv3:",conv3.shape)
    conv4 = tf.nn.relu(conv2d(conv3, weights['W_conv4']) + biases['bc4'])
    print("conv3:",conv3.shape)
    #conv4 = conv3
    conv4 = maxpool2d(conv4)
    print("maxpool:",conv4.shape)
#     conv4 = tf.nn.dropout(conv4, 0.25)
    
    conv5 = tf.nn.relu(conv2d(conv4, weights['W_conv5']) + biases['bc5'])
    print("conv5:",conv5.shape)
    conv6 = tf.nn.relu(conv2d(conv5, weights['W_conv6']) + biases['bc6'])
    print("conv6:",conv6.shape)
    #conv6 = conv5
    conv6 = maxpool2d(conv6)
    print("conv6:",conv6.shape)
#     conv6 = tf.nn.dropout(conv6, 0.25)

    fc1 = tf.reshape(conv6,[-1, weights['W_fc1'].get_shape().as_list()[0]])
    fc1 = tf.nn.relu(tf.matmul(fc1, weights['W_fc1'])+biases['b_fc1'])
    print("fc1:",fc1.shape)
#     fc1 = tf.nn.dropout(fc1, 0.3)
    
    fc2 = tf.nn.relu(tf.matmul(fc1, weights['W_fc2'])+biases['b_fc2'])
#     fc2 = tf.nn.dropout(fc2, 0.3)  
    #fc2 = fc1
    
#     out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    print("fc2:",fc2.shape)
    print(fc2)
    
#     x_norm = batch_norm_wrapper(fc2,is_training)
    
    
#     inputs=fc2
#     scale = tf.Variable(tf.ones([inputs.get_shape()[-1]]))
#     beta = tf.Variable(tf.zeros([inputs.get_shape()[-1]]))
#     pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]), trainable=False)
#     pop_var = tf.Variable(tf.ones([inputs.get_shape()[-1]]), trainable=False)
#     decay=0.9999

#     if is_training:
#         batch_mean, batch_var = tf.nn.moments(inputs,[0])
#         train_mean = tf.assign(pop_mean,
#                                pop_mean * decay + batch_mean * (1 - decay))
#         train_var = tf.assign(pop_var,
#                               pop_var * decay + batch_var * (1 - decay))
#         with tf.control_dependencies([train_mean, train_var]):
#             return tf.nn.batch_normalization(inputs,batch_mean, batch_var, beta, scale, epsilon)
#     else:
#         return tf.nn.batch_normalization(inputs,pop_mean, pop_var, beta, scale, epsilon)
    
    
#     x_norm = tf.layers.batch_normalization(fc2, training=True)
#     input_gru = tf.repeat(fc2,)
    
#     print(x_norm.shape)
    return fc2

# def Gru(hidden_size):  
#     gru = GRU(1024,hidden_size)

#     W_output = tf.Variable(tf.truncated_normal(dtype=tf.float64, shape=(hidden_size, 1), mean=0, stddev=0.01))
#     b_output = tf.Variable(tf.truncated_normal(dtype=tf.float64, shape=(1,), mean=0, stddev=0.01))
#     output = tf.map_fn(lambda h_t: tf.matmul(h_t, W_output) + b_output, gru.h_t)

#     return output

In [8]:
# import torch.utils.data as data
import cv2
import sys
from os import listdir
from os.path import join
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

def resize_img(png_file_path):
        img_rgb = cv2.imread(png_file_path)
        img_grey = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
        img_adapted = cv2.adaptiveThreshold(img_grey, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 101, 9)
        img_stacked = np.repeat(img_adapted[...,None],3,axis=2)
        resized = cv2.resize(img_stacked, (224,224), interpolation=cv2.INTER_AREA)
        bg_img = 255 * np.ones(shape=(224,224,3))
#         print(bg_img.shape,resized.shape)
        bg_img[0:224, 0:224,:] = resized
        bg_img /= 255
        bg_img = np.rollaxis(bg_img, 2, 0)  
#         print(bg_img.shape)
        return bg_img
    
def load_doc(filename):
    file = open(filename, 'r',encoding='UTF-8')
    text = file.read()
    file.close()
    return text

class Dataset():
    def __init__(self, data_dir, input_transform=None, target_transform=None):
        self.data_dir = data_dir
        self.image_filenames = []
        self.texts = []
        all_filenames = listdir(data_dir)
        all_filenames.sort()
        for filename in (all_filenames):
            if filename[-3:] == "png":
                self.image_filenames.append(filename)
            else:
                text = '<START> ' + load_doc(self.data_dir+filename) + ' <END>'
                text = ' '.join(text.split())
                text = text.replace(',', ' ,')
                self.texts.append(text)
        self.input_transform = input_transform
        self.target_transform = target_transform
        
        # Initialize the function to create the vocabulary 
        tokenizer = Tokenizer(filters='', split=" ", lower=False)
        # Create the vocabulary 
        tokenizer.fit_on_texts([load_doc('vocabulary.vocab')])
        self.tokenizer = tokenizer
        # Add one spot for the empty word in the vocabulary 
        self.vocab_size = len(tokenizer.word_index) + 1
        # Map the input sentences into the vocabulary indexes
        self.train_sequences = tokenizer.texts_to_sequences(self.texts)
        # The longest set of boostrap tokens
        self.max_sequence = max(len(s) for s in self.train_sequences)
        # Specify how many tokens to have in each input sentence
        self.max_length = 48
        
        X, y, image_data_filenames = list(), list(), list()
        for img_no, seq in enumerate(self.train_sequences):
            in_seq, out_seq = seq[:-1], seq[1:]
            out_seq = to_categorical(out_seq, num_classes=self.vocab_size)
            image_data_filenames.append(self.image_filenames[img_no])
            X.append(in_seq)
            y.append(out_seq)
                
        self.X = X
        self.y = y
        self.image_data_filenames = image_data_filenames
        self.images = list()
        for image_name in self.image_data_filenames:
            image = resize_img(self.data_dir+image_name)
            self.images.append(image)

In [9]:
dir_name = 'all_data5/'
batch_size = 32
my_dateset = Dataset(dir_name)

In [10]:
x_train = np.array(my_dateset.images,dtype=np.float32)
for i in range(len(x_train)):
    x_train[i]=np.array(x_train[i],dtype=np.float32)
print(x_train.shape)
# batch_size = 128
# dataset = tf.data.Dataset.from_tensor_slices((x))
# iterator = dataset.repeat().batch(batch_size).make_initializable_iterator()
# data_batch = iterator.get_next()

(100, 3, 224, 224)


In [11]:
im = tf.placeholder(dtype=tf.float32, shape=(None,3,224,224), name='im')
# is_training = tf.placeholder(dtype=tf.bool, name="is_training")
model1 = cnn(im,weights,biases)
output_train = batch_norm_wrapper(model1,True)
output_test = batch_norm_wrapper(model1,False)

in cnn
-1
00
bef (?, 3, 224, 224)
aft (?, 224, 224, 3)
0
******** <tf.Variable 'W0:0' shape=(3, 3, 3, 32) dtype=float32_ref>
1
conv1: (?, 224, 224, 32)
2
conv2: (?, 224, 224, 32)
3
maxpool: (?, 112, 112, 32)
okay
conv3: (?, 112, 112, 64)
conv3: (?, 112, 112, 64)
maxpool: (?, 56, 56, 64)
conv5: (?, 56, 56, 128)
conv6: (?, 56, 56, 128)
conv6: (?, 28, 28, 128)
fc1: (?, 1024)
fc2: (?, 50)
Tensor("Relu_7:0", shape=(?, 50), dtype=float32)


In [108]:
gru = GRU_cell(100,256,18)
hidden_size=256

# W_output = tf.Variable(tf.truncated_normal(dtype=tf.float64, shape=(hidden_size, 18), mean=0, stddev=0.01),trainable=True)
# b_output = tf.Variable(tf.truncated_normal(dtype=tf.float64, shape=(18,), mean=0, stddev=0.01),trainable=True)

# W_output = weights['out']
# b_output = biases['out']

# output = tf.map_fn(lambda h_t: tf.matmul(h_t, W_output) + b_output, gru.h_t)

# output1 = tf.nn.relu(tf.matmul(gru.h_t,W_output)+b_output)
# out2 = tf.matmul(gru.h_t[0], W_output)+b_output

# tf.get_variable('W7', shape=(1024,50), initializer=tf.contrib.layers.xavier_initializer())

# out3 = gru.h_t
# outs = gru.outputs
print(gru.processed_input.shape)
# print(gru.images.shape)


output = gru.get_outputs()
# hidden = gru.get_states()
# print(output.shape)
print("full:",output.shape)
ol = gru.get_last_output()
print("ol:",ol.shape)
# print(hidden.shape)
# print(gru.h_t.shape)
# print(out3.shape)
# print(gru.outputs.shape)

(?, ?, 50)
%% (?, 50) (?, 256)
(?, 1, 50) (1, 1, 50)
embed ip: (1, 1, 100)
all_ht: (?, ?, 256)
op: (?, 18)
all_op: (?, ?, 18)
full: (?, ?, 18)
%% (?, 50) (?, 256)
(?, 1, 50) (1, 1, 50)
embed ip: (1, 1, 100)
all_ht: (?, ?, 256)
op: (?, 18)
ol: (?, 18)


In [13]:
y_train = my_dateset.X
y_train=np.array(y_train)
        
for i in range(len(y_train)):
    y_train[i]=np.array(y_train[i])
#     for j in range(len(x1[i])):
#         x1[i][j]=np.array(x1[i][j])
print(y_train.shape)
print(y_train[0].shape)
    

# x1=tf.constant(x1[0])
print("hi")
            
VOCAB_LEN=19
EMBED_SIZE=50
embeddings = tf.Variable(tf.random_uniform([VOCAB_LEN, EMBED_SIZE]))
caption_p = tf.placeholder(dtype=tf.int32, shape=(None,None), name='caption_p')
embed = tf.nn.embedding_lookup(embeddings, caption_p)

(100,)
(59,)
hi


In [14]:
expected = my_dateset.y
expected=np.array(expected)
for e in range(len(expected)):
    expected[e]=np.array(expected[e])
print(expected.shape)

(100,)


In [112]:
expected_output = tf.placeholder(dtype=tf.float32, shape=(None,None,None), name='expected_output')
loss = tf.reduce_sum(tf.squared_difference(ol ,expected_output)) #/ float(1)
# e_loss = tf.placeholder(dtype=tf.float64,name='e_loss')
# epoch_loss = tf.reduce_sum(e_loss)
# train2 = tf.train.AdamOptimizer(0.0001).minimize(tf.reduce_sum(e_loss))
# train_step = tf.train.AdamOptimizer().minimize()
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

In [16]:
import functools 
def pad(batch_y):
    print(batch_y.shape)
    x=0
    for y in batch_y:
        if(len(y)>x):
            x=len(y)
#     x = functools.reduce(lambda x,y: len(x) if(len(x)>len(y)) else len(y),batch_y)
    
    ret = []
    for y in range(len(batch_y)):
        res=np.zeros(x)
        s = batch_y[y]
        res[0:len(s)]=batch_y[y]
#         batch_y[y]=res
        ret.append(res)
    return np.array(ret)
        
        
# a=[[1,2],[1,2,3]]
# pad(a)

def pad2(batch_ex):
#     r = functools.reduce(lambda x,y: len(x) if(len(x)>len(y)) else len(y),batch_ex)
#     print(":::::",r)
    r=0
    c=0
    for ex in batch_ex:
        shape = ex.shape
#         print(shape)
        if(shape[0]>r):
            r=shape[0]
        if(shape[1]>c):
            c=shape[1]
#     c = functools.reduce(lambda x,y: len(x[0]) if(len(x[0])>len(y[0])) else len(y[0]),batch_ex)
#     print(":::::",c)
#     print(r,c)
    ret=[]
    for ex in batch_ex:
        res=np.zeros((r,c))
#         print(res.shape)
#         print(ex.shape)
        res[0:ex.shape[0],0:ex.shape[1]]=ex
        ret.append(res)
#     print(ret)
        
    return(np.array(ret))

In [113]:
epoch = 5
vocab_size = 19
batch_size=1

x_train = my_dateset.images
caption = my_dateset.X
expected = my_dateset.y

saver = tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    loss_ar=[]
    for e in range(epoch):
        loss_no=[]
        print(e)
        for batch in range(len(x_train)//batch_size):
            print("batch ",batch)
            batch_x = x_train[batch*batch_size:min((batch+1)*batch_size,len(x_train))]
            batch_y = caption[batch*batch_size:min((batch+1)*batch_size,len(caption))] 
            batch_ex = expected[batch*batch_size:min((batch+1)*batch_size,len(expected))]
            print("x:",len(batch_x))
            print("y:",len(batch_y))
            print("ex:",len(batch_ex))
            
#             print(batch_y)
            
            batch_x = np.array(batch_x)
            for b in range(len(batch_x)):
                batch_x[b]=np.array(batch_x[b])
            batch_y = np.array(batch_y)
            for b in range(len(batch_y)):
                batch_y[b]=np.array(batch_y[b])
            batch_ex = np.array(batch_ex)
            for b in range(len(batch_ex)):
                batch_ex[b]=np.array(batch_ex[b])
                
            print("bex:",batch_ex.shape)
#             print(batch_ex[0].shape)
                
            batch_y = pad(batch_y)
            batch_ex = pad2(batch_ex)
            
#             batch_y = batch_y.reshape((-1,1))
#             print(batch_ex.shape)
            
            sess.run(init)
            # image = K.expand_dims(x[i], 0)
            # print("im:",image.shape)
            c,m = sess.run([output_train,model1],feed_dict={im:batch_x})
            print("c:",c)
            
#             x_norm = batch_norm_wrapper(c,True,)
            
#             x_norm = tf.layers.batch_normalization(c, training=True)
#             x = sess.run(x_norm)
#             print(x.shape)
#             # em = K.expand_dims(x1[i], 0)

#             inp2 = sess.run(embed,feed_dict={caption_p:batch_y})
#             print("inp2:",inp2.shape)

#             features_try = K.tile(K.expand_dims(c, 1), [1, K.shape(inp2)[1], 1])
#             print("ktile:",features_try.shape)
            embeddings = tf.concat([features_try,inp2],2)
            print("emb:",embeddings.shape)
            inp = sess.run(embeddings)

            a = sess.run(ol,feed_dict={gru.images:c,gru.words:batch_y})
            print("a:",a.shape,"\n")

            # print("Sssss:",ex.shape)
            ls,tr = sess.run([loss,train_step],feed_dict ={expected_output:batch_ex,gru.images:c,gru.words:batch_y})
            print(ls/batch_size)
            loss_no.append(ls/batch_size)
            print("\n\n")
#         el = sess.run(epoch_loss,feed_dict={e_loss:loss_no})
        loss_ar.append(loss_no)

        print("-----------------------------------------------------------------") 
    save_path = saver.save(sess, "model.ckpt")

0
batch  0
x: 1
y: 1
ex: 1
bex: (1, 59, 18)
(1, 59)
c: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
emb: (10, 94, 100)
a: (1, 18) 

1006.5386352539062



batch  1
x: 1
y: 1
ex: 1
bex: (1, 94, 18)
(1, 94)
c: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
emb: (10, 94, 100)
a: (1, 18) 

1598.0025634765625



batch  2
x: 1
y: 1
ex: 1
bex: (1, 76, 18)
(1, 76)
c: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
emb: (10, 94, 100)
a: (1, 18) 

1291.0029296875



batch  3
x: 1
y: 1
ex: 1
bex: (1, 26, 18)
(1, 26)


KeyboardInterrupt: 

In [18]:
print(loss_ar)

[[82.2716421177868, 92.81862903434791, 96.30940812570955, 84.75820456588983, 86.95217646124553, 78.29988489684754, 102.29989123686144, 98.14264792282572, 89.81742759596796, 110.03864119264809], [119.272947928158, 97.48594405296787, 107.28251140724505, 106.16270920030271, 109.26193702470599, 103.9932880860052, 83.27586920555275, 65.61239812023724, 103.1262389559671, 100.50055311180877], [107.35299611391278, 94.20644692274458, 92.5262013955177, 98.51224381989267, 84.46880247133484, 97.11596637441814, 86.22528406998772, 73.61804806208723, 89.15349053332109, 123.26068964406713], [99.53647862179639, 91.7311697680406, 84.16802474789583, 107.03807201975133, 66.93254981303151, 78.67394327839861, 102.06400343448196, 96.05603672002141, 95.07008435066464, 99.2345743383708], [105.76051951369845, 99.5854322749878, 84.15821082562589, 92.1515492042857, 89.92220258869473, 83.75604667947928, 99.69011185893811, 83.38375060660101, 89.92259518919097, 87.8045557599808]]


In [19]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None
def load_val_images(data_dir):
    image_filenames =[]
    images = []
    all_filenames = listdir(data_dir)
    all_filenames.sort()
    for filename in (all_filenames):
        if filename[-3:] == "png":
            image_filenames.append(filename)
    for name in image_filenames:
        image = resize_img(data_dir+name)
        images.append(image)
    return images

In [20]:
decoded_words = []
star_text = '<START> '
image = load_val_images('all_data6/')[0]
img_tensor=np.expand_dims(np.array(image),0)
img_tensor=np.array(img_tensor)
''''image = Variable(torch.FloatTensor([image]))'''
predicted = '<START> '
 
img_tensor.shape

(1, 3, 224, 224)

In [21]:
predicted='<START>'
with tf.Session() as sess:
#     init = tf.global_variables_initializer()
#     sess.run(init)
    saver.restore(sess, "model.ckpt")
#     print("####",weights['W_conv1'].eval())
    for di in range(50):
        sequence = my_dateset.tokenizer.texts_to_sequences([star_text])[0]
#         print(sequence)
        decoder_input = np.array(sequence).reshape(-1,1)
#         c = sess.run(model1,feed_dict={im:img_tensor})
#         print("--------------------------")
        c,m = sess.run([output_test,model1],feed_dict={im:img_tensor})
#         print(di, c)
#         print("--------------------------")
        
        inp2 = sess.run(embed,feed_dict={caption_p:decoder_input})
        print(inp2)
        features_try = K.tile(K.expand_dims(c, 1), [1, K.shape(inp2)[1], 1])
        embeddings = tf.concat([features_try,inp2],2)
        inp = sess.run(embeddings)
        a = sess.run(output1,feed_dict={gru.input_layer:inp})
        data=list(a[0][0])
        i=data.index(max(data))
        word = word_for_id(i,my_dateset.tokenizer)
        if word is None:
            continue
        predicted += word + ' '
        star_text = word
        print(predicted)
        if word == '<END>':
            break

INFO:tensorflow:Restoring parameters from model.ckpt
<START>text 
<START>text text 
<START>text text text 
<START>text text text text 
<START>text text text text text 
<START>text text text text text text 
<START>text text text text text text text 
<START>text text text text text text text text 
<START>text text text text text text text text text 
<START>text text text text text text text text text text 
<START>text text text text text text text text text text text 
<START>text text text text text text text text text text text text 
<START>text text text text text text text text text text text text text 
<START>text text text text text text text text text text text text text text 
<START>text text text text text text text text text text text text text text text 
<START>text text text text text text text text text text text text text text text text 
<START>text text text text text text text text text text text text text text text text text 
<START>text text text text text text text text